<a href="https://colab.research.google.com/github/kkrizek06/Amazon_Vine_Analysis/blob/main/vine_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

In [5]:
vote_df = df.filter(df.total_votes >= 20)
vote_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44783596|R31EUR60FV3BU5|B00ZGPZR9I|     514136181|  Wodehouse in Exile|       Video DVD|          4|           32|         39|   N|                N|Unfair Accusation...|Amazed after Wode...| 2015-08-31|
|         US|   20714119|R12Q1NO1HI9PP4|B00XUV1B4U|     881344665|Mad Max Anthology...|       Video DVD|          1|    

In [6]:
new_df = vote_df.filter((vote_df.helpful_votes / vote_df.total_votes) *100 >= 50)
new_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44783596|R31EUR60FV3BU5|B00ZGPZR9I|     514136181|  Wodehouse in Exile|       Video DVD|          4|           32|         39|   N|                N|Unfair Accusation...|Amazed after Wode...| 2015-08-31|
|         US|   12720421|R3MDX24QDAT0OW|B00ZCHTQGA|     779817400|Northmen - A Viki...|       Video DVD|          4|    

In [7]:
Y_df = new_df.filter(new_df.vine =='Y')
Y_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27249597|R3PBQ665MBL8CP|B00E1LT41K|     592331046|The White Queen: ...|       Video DVD|          3|           66|         72|   Y|                N|An enjoyable watc...|I enjoyed this TV...| 2014-03-27|
|         US|   37876368|R3EMD30K7MNOZ0|B00A27OMKU|     678954891|Shiva Rea: Yoga i...|       Video DVD|          2|    

In [8]:
N_df = new_df.filter(new_df.vine =='N')
N_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44783596|R31EUR60FV3BU5|B00ZGPZR9I|     514136181|  Wodehouse in Exile|       Video DVD|          4|           32|         39|   N|                N|Unfair Accusation...|Amazed after Wode...| 2015-08-31|
|         US|   12720421|R3MDX24QDAT0OW|B00ZCHTQGA|     779817400|Northmen - A Viki...|       Video DVD|          4|    

In [20]:
totalY = Y_df.count()
totalY


49

In [21]:
Yfive_star = Y_df.filter(Y_df.star_rating == 5).count()
Yfive_star


9

In [22]:
Yfive_star_percent = (Yfive_star / totalY) *100
Yfive_star_percent

18.367346938775512

In [23]:
totalN = N_df.count()
totalN


151400

In [24]:
Nfive_star = N_df.filter(N_df.star_rating == 5).count()
Nfive_star

78061

In [25]:
Nfive_star_percent = (Nfive_star / totalN) *100
Nfive_star_percent

51.55944517833554